# Anomaly Detection

In this notebook, we explore various techniques for out-of-distribution (OOD) detection. We will be using a pre-trained ResNet on the CIFAR-10 dataset. We will then use images from other datasets as examples of OOD datapoints. We will compare different OOD detection methods using their AUROC scores.

## Pre-requisites

In the following cells, we are installing and importing the necessary libaries and downloading the classification model.

In [ ]:
!wget https://github.com/hendrycks/outlier-exposure/raw/master/CIFAR/snapshots/baseline/cifar10_wrn_baseline_epoch_99.pt
!wget https://raw.githubusercontent.com/hendrycks/pre-training/master/robustness/adversarial/models/wrn_with_pen.py
!pip3 install torchvision==0.12.0

--2022-08-09 16:14:45--  https://github.com/hendrycks/outlier-exposure/raw/master/CIFAR/snapshots/baseline/cifar10_wrn_baseline_epoch_99.pt
Resolving github.com (github.com)... 13.114.40.48
Connecting to github.com (github.com)|13.114.40.48|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hendrycks/outlier-exposure/master/CIFAR/snapshots/baseline/cifar10_wrn_baseline_epoch_99.pt [following]
--2022-08-09 16:14:46--  https://raw.githubusercontent.com/hendrycks/outlier-exposure/master/CIFAR/snapshots/baseline/cifar10_wrn_baseline_epoch_99.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9037421 (8.6M) [application/octet-stream]
Saving to: ‘cifar10_wrn_baseline_epoch_99.pt’

cifar10_wrn_baselin 10

In [ ]:
import torch
import math
from torch.utils.data import Dataset
import torchvision
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
from torchvision import datasets
import numpy as np
import torch.nn.functional as F
import sklearn.metrics as sk
from wrn_with_pen import WideResNet  # Loads pre-trained model

prefetch = 2

## Model and Data Loading

In the following cells, we are loading the CIFAR-10 model. Additionally, we are retrieving CIFAR-10 and out-of-distribution datasets.

In [ ]:
# Load CIFAR-10 model

net = WideResNet(depth=40, num_classes=10, widen_factor=2, dropRate=0.3)

if torch.cuda.is_available():
    net.load_state_dict(torch.load('cifar10_wrn_baseline_epoch_99.pt'))
    net.eval()
    net.cuda()
else:
    net.load_state_dict(torch.load('cifar10_wrn_baseline_epoch_99.pt', map_location=torch.device('cpu')))
    net.eval()

In [ ]:
# /////////////// Loading Datasets ///////////////
mean = [x / 255 for x in [125.3, 123.0, 113.9]]
std = [x / 255 for x in [63.0, 62.1, 66.7]]

# /////////////// CIFAR-10 ///////////////

data_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean, std)])

cifar_10_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=data_transform
)

cifar10_data = cifar_10_data
cifar10_loader = torch.utils.data.DataLoader(cifar10_data, batch_size=200, shuffle=False,
                                          num_workers=prefetch, pin_memory=True)
ood_num_examples = len(cifar10_data) // 5

# /////////////// Rademacher Noise ///////////////

dummy_targets = torch.ones(ood_num_examples)
ood_data = torch.from_numpy(np.random.binomial(
    n=1, p=0.5, size=(ood_num_examples, 3, 32, 32)).astype(np.float32)) * 2 - 1
rademacher_ood_data = torch.utils.data.TensorDataset(ood_data, dummy_targets)
rademacher_ood_loader = torch.utils.data.DataLoader(rademacher_ood_data, batch_size=200, shuffle=True)

# /////////////// SVHN /////////////// (Street View House Numbers)

data_transform = transforms.Compose([transforms.Resize(32), transforms.ToTensor(), transforms.Normalize(mean, std)])

svhn_ood_data = torchvision.datasets.SVHN(root = "data", 
                          split="test",
                          transform = data_transform, 
                          download = True)

svhn_ood_loader = torch.utils.data.DataLoader(svhn_ood_data, batch_size=200, shuffle=True,
                                         num_workers=prefetch, pin_memory=True)

data_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean, std)])

cifar100_ood_data = datasets.CIFAR100(
    root="data",
    train=False,
    download=True,
    transform=data_transform
)

cifar100_ood_loader = torch.utils.data.DataLoader(cifar100_ood_data, batch_size=200, shuffle=True,
                                         num_workers=prefetch, pin_memory=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data


  0%|          | 0/64275384 [00:00<?, ?it/s]

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting data/cifar-100-python.tar.gz to data


## General Functions

The following cells define functions that we will use in the rest of the code.

In [ ]:
concat = lambda x: np.concatenate(x, axis=0)
to_np = lambda x: x.data.cpu().numpy()

'''
Calculates the anomaly scores for a portion of the given dataset. 
If a GPU is not available, will run on a smaller fraction of the
dataset, so that calculations will be faster.

loader: A DataLoader that contains the loaded data of a dataset
anomaly_score_calculator: A function that takes in the output 
                          logit of a batch of data and/or the 
                          penultimate.
model_net: The classifier model.
usePenultimate: True if anomaly_score_calculator needs the 
                penultimate as a parameter. False otherwise.
'''
def get_ood_scores(loader, anomaly_score_calculator, model_net, use_penultimate = False):
    _score = []

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(loader):
            if torch.cuda.is_available():
                fraction = 200
            else:
                fraction = 1000
            if batch_idx >= ood_num_examples // fraction:
                break

            if torch.cuda.is_available():
                data = data.cuda()

            output = model_net(data)

            if use_penultimate:
                score = anomaly_score_calculator(output[0], output[1])
            else:
                score = anomaly_score_calculator(output[0])
            _score.append(score)

    return concat(_score).copy()

In [ ]:
# /////////////// Printing Results ///////////////
all_anomaly_results = {}

'''
Returns and prints out the AUROC score of a dataset.

ood_loader: A DataLoader that contains the loaded data of a dataset
anomaly_score_calculator: A function that takes in the output 
                          logit of a batch of data and/or the 
                          penultimate.
model_net: The classifier model.
usePenultimate: True if anomaly_score_calculator needs the 
                penultimate as a parameter. False otherwise.
'''
def get_and_print_results(ood_loader, anomaly_score_calculator, model_net, use_penultimate):
    out_score = get_ood_scores(ood_loader, anomaly_score_calculator, model_net, use_penultimate = use_penultimate)
    auroc = get_auroc(out_score, in_score)
    print('AUROC: \t\t\t{:.2f}'.format(100 * auroc) + "%")
    return auroc

'''
Prints out the AUROC score of all the OOD datasets. The results 
will be appended to global variable all_anomaly_results, which 
is used later for display purposes.

anomaly_score_calculator: A function that takes in the output 
                          logit of a batch of data and/or the 
                          penultimate.
anomaly_score_name: The name of the anomaly score method.
model_net: The classifier model.
model_name: The name of the classifier model.
usePenultimate: True if anomaly_score_calculator needs the 
                penultimate as a parameter. False otherwise.
'''
def print_all_results(anomaly_score_calculator, anomaly_score_name, model_net, model_name = "default_model", use_penultimate = False):
    global in_score, all_anomaly_results
    in_score = get_ood_scores(cifar10_loader, anomaly_score_calculator, model_net, use_penultimate)
    results = []

    print('Rademacher Noise Detection')
    auroc = get_and_print_results(rademacher_ood_loader, anomaly_score_calculator, model_net, use_penultimate)
    results.append(auroc)

    print('\nSVHN Detection')
    auroc = get_and_print_results(svhn_ood_loader, anomaly_score_calculator, model_net, use_penultimate)
    results.append(auroc)

    print('\nCIFAR-100 Detection')
    auroc = get_and_print_results(cifar100_ood_loader, anomaly_score_calculator, model_net, use_penultimate)
    results.append(auroc)

    average = sum(results) / len(results)
    results.append(average)

    if not model_name in all_anomaly_results:
        all_anomaly_results[model_name] = {}
    all_anomaly_results[model_name][anomaly_score_name] = results

## Implement AUROC Score
Fill in the get_auroc score. We will use this function in order to calculate the AUROC score of an out-of-distribution dataset.

It may be helpful to use the sklearn.metrics.roc_auc_score() function. Both _pos and _neg should be used.

In [ ]:
import sklearn

'''
Calculates the AUROC score of a OOD dataset.

_pos: an array of anomoly scores of the OOD dataset
_neg: an array of anomoly scores of images in the CIFAR-10 dataset
return: The AUROC score the data in decimal form
'''
def get_auroc(_pos, _neg):
    _pos = torch.from_numpy(_pos); _neg = torch.from_numpy(_neg)
    labels = torch.vstack((torch.ones_like(_pos).to(torch.bool), torch.zeros_like(_neg).to(torch.bool)))
    scores = torch.vstack((_pos,_neg))

    auroc_score = sklearn.metrics.roc_auc_score(labels, scores)

    return auroc_score

## Implement Anomaly Score Calculators
Fill in the folllowing functions which calculate the anomaly score given the model's output for a batch of data (the output will contain one logit per image in the batch).

The following equations show how the logits should be transformed in order to get the anomaly score.

Max Logit Equation: <br>
$\text{Score}=-\text{max} l_k$

Max Softmax Equation: <br>
$\text{Score}=-\text{max} p(y=k|x)$

Cross Entropy Anomaly Equation: <br>
$\text{Score} = \bar{l}-\text{log}∑_{c=1}^{\text{num\_classes}}e^{l_c}$

In [ ]:
# /////////////// Anomaly Score Calculators ///////////////

'''
Calculates the max logit anomaly score of a batch of outputs. Return tensor shape (N,)

output: The model's output for a batch of data. Shape (N,C)
'''
def max_logit_anomaly_score(output):
    anomaly_scores = (-1) * torch.max(output, dim=1).values
    return to_np(anomaly_scores)

'''
Calculates the max softmax anomaly score of a batch of outputs. Return tensor shape (N,)

output: The model's output for a batch of data. Shape (N,C)
'''
def max_softmax_anomaly_score(output):
    softmax = torch.nn.Softmax(dim=1)
    anomaly_scores = (-1) * torch.max(softmax(output), dim=1).values
    return to_np(anomaly_scores)

'''
Calculates the cross entropy anomaly score of a batch of outputs.

output: The model's output for a batch of data.
'''
def cross_entropy_anomaly_score(output):
    C = output.shape[1]  # number of classes

    log_sum_e = torch.log(torch.sum(torch.exp(output), dim=1))
    avg_l = torch.sum(output, dim=1) / C

    anomaly_scores = avg_l - log_sum_e

    return to_np(anomaly_scores)

## Print AUROC Results

Here we see how well each of the anomaly score calculators do on the OOD datasets. (We are using the other datasets as OOD images for CIFAR-10)

In [ ]:
print("======= Max Logit AUROC Scores =======")
print_all_results(max_logit_anomaly_score, "Max Logit", net)

======= Max Logit AUROC Scores =======
Rademacher Noise Detection
AUROC: 			70.85%

SVHN Detection
AUROC: 			90.40%

CIFAR-100 Detection
AUROC: 			87.45%


In [ ]:
print("======= Max Softmax AUROC Scores =======")
print_all_results(max_softmax_anomaly_score, "Max Softmax", net)

======= Max Softmax AUROC Scores =======
Rademacher Noise Detection
AUROC: 			80.25%

SVHN Detection
AUROC: 			92.50%

CIFAR-100 Detection
AUROC: 			89.25%


In [ ]:
print("======= Cross Entropy AUROC Scores =======")
print_all_results(cross_entropy_anomaly_score, "Cross Entropy", net)

======= Cross Entropy AUROC Scores =======
Rademacher Noise Detection
AUROC: 			70.50%

SVHN Detection
AUROC: 			90.55%

CIFAR-100 Detection
AUROC: 			86.65%


## Implement ViM

We will now implement virtual-logit matching (ViM). ViM works by doing the following. 

ViM calculates the **principal space** ($P^\perp$) using the training data's (CIFAR-10)  feature (penultimate). Let us use the 12 most significant principal components for our principal space.

Then, ViM calculates **alpha** ($\alpha$) by projecting the training data's features onto the prinicpal space calculated before. This is known as the **residual** ($x^{P^\perp}$). Alpha is then calculated by the following equation:
> $\alpha := \frac{∑^K_{i=1}\text{max}_{j=1,...,C}\{l^i_j\}}{∑^K_{i=1}\| x_i^{P\perp} \|}$

In other words, alpha is the sum of each logit's maximum value, divided by the sum of the norms of each feature's residuals. Here K is the # of datapoints in the batch used for finding the principal space, and l_i,j is j-th logic of data point i.

The alpha value helps adjust the OOD logit score. If you rearrange the equation by multiplying the denominator on both sides, alpha is set such that when multiplied with the sum of the OOD logit scores, it equals the sum of maximum logit over the entire dataset.

In [ ]:
# You may find the following functions useful.
from numpy.linalg import pinv, norm
from scipy.special import logsumexp

# We'll be using the PCA module from sklearn. Under the hood, it uses SVD
# Note, the module centers (but not scales) the data before performing PCA, and will center the subsequent input data before giving the transformed coordinates
from sklearn.decomposition import PCA

_score = []
to_np = lambda x: x.data.cpu().numpy()
concat = lambda x: np.concatenate(x, axis=0)

# Extraction fully connected layer's weights and biases
w, b = net.fc.weight.cpu().detach().numpy(), net.fc.bias.cpu().detach().numpy()
# Origin of a new coordinate system of feature space to remove bias
u = -np.matmul(pinv(w), b)

'''
Calculates and returns the principal space and alpha values given
the training data the model used.

training_data_loader: A DataLoader that contains the loaded data of a 
                      training dataset.
model_net: The classifier model.
verbose: If true, will print out the alpha value.
return: Returns a tuple whose first element is the principal space matrix
        and second element is the alpha value.
'''
def compute_ViM_principal_space_and_alpha(training_data_loader, model_net, verbose = False):
    result = []

    # Getting the first batch of the training data to calculate principal space and alpha
    training_data, target = next(iter(training_data_loader))
    if torch.cuda.is_available():
        training_data = training_data.cuda()

    result = model_net(training_data)
    logit = result[0] # Logits (values before final softmax for output), shape (N,C)
    penultimate = result[1] # Features/Penultimate (values before fully connected layer), shape (N,D)

    # Turn to numpy arrays
    logit_id_train = logit.cpu().detach().numpy().squeeze()
    feature_id_train = penultimate.cpu().detach().numpy().squeeze()

    # Values to calculate
    principal_space = alpha = None

    print(logit_id_train.shape)

    if verbose:
        print('Computing principal space...')

    pca = PCA(n_components=12)
    pca.fit(feature_id_train)
    principal_space = pca

    if verbose:
        print('Computing alpha...')

    feature_transformed = pca.transform(feature_id_train)  # Shape (N,12)
    p_components = pca.components_  # Shape (12,D)
    residuals = feature_transformed @ p_components  # Shape (N,D)
    sum_norm_residuals = torch.sum(torch.norm(torch.from_numpy(residuals), dim=1))

    max_logits = torch.max(logit, dim=1).values
    sum_max_logits = torch.sum(max_logits)

    alpha = sum_max_logits / sum_norm_residuals

    if verbose:
        print(f'alpha = {alpha}')

    return principal_space, alpha

principal_space, alpha = compute_ViM_principal_space_and_alpha(cifar10_loader, net, verbose = True)

(200, 10)
Computing principal space...
Computing alpha...
alpha = 2.25811505317688


## Implement ViM Anomaly Score Calculator

Now, we'll implement the ViM anomaly score calculator. 

First, we want to project our penultimate/feature values onto the principal space which we found before, which is called the residual. We multiply the norm of this residual by alpha to get what we call the **virtual logit score**. 

Next, we get what we call the **energy score** by taking LogSumExp of the logits. 

Finally, the outputted **anomaly score** is calculated by subtracting the virtual logit by the energy score.

$\text{vlogit}= \alpha \| {x^{P^\perp}} \|$\
$\text{energy} = \text{ln}\sum_{i=1}^C e^{l_i}$\
$\text{anomaly\_score} = \text{vlogit} - \text{energy}$

In [ ]:
'''
Calculates the ViM anomaly score of a batch of outputs.

output: The model's output for a batch of data. Shape (N,C)
penultimate: The model's penultimate (feature values) for a batch of data. Shape (N,D)
'''
def ViM_anomaly_score_calculator(output, penultimate):
    logit_id_val = output.cpu().detach().numpy().squeeze()
    feature_id_val = penultimate.cpu().detach().numpy().squeeze()

    pca = principal_space # Shorten name
    features_transformed = pca.transform(feature_id_val)  # Shape (N,12)
    p_components = pca.components_  # Shape (12,D)
    residuals = features_transformed @ p_components  # Shape (N,D)
    norm_residuals = torch.norm(torch.from_numpy(residuals), dim=1)
    vlogits = alpha.item() * norm_residuals

    energies = logsumexp(logit_id_val, axis=1)

    score_id = vlogits - energies
    
    return score_id

print("======= ViM_anomaly_score_calculator =======")
w, b = net.fc.weight.cpu().detach().numpy(), net.fc.bias.cpu().detach().numpy()
u = -np.matmul(pinv(w), b)
principal_space, alpha = compute_ViM_principal_space_and_alpha(cifar10_loader, net) # Making sure you have the correct ViM values before calculating the score 
print_all_results(ViM_anomaly_score_calculator, "ViM", net, use_penultimate = True)

======= ViM_anomaly_score_calculator =======
(200, 10)
Rademacher Noise Detection
AUROC: 			95.95%

SVHN Detection
AUROC: 			93.25%

CIFAR-100 Detection
AUROC: 			83.50%


## Compare Anomaly Score Results
Run the following cell to see how the different anomaly score calculators compare to each other for the OOD datasets. You should see that ViM is superior to other anomaly scores in all of the datasets.

In [ ]:
# ///////////////// Compare Results /////////////////

def get_results_max(model_name = "normal"):
    all_anomaly_results[model_name]["max"] = [0,0,0,0,0]
    for key in all_anomaly_results[model_name].keys():
        if (key != "max"):
            index = 0
            for score in all_anomaly_results[model_name][key]:
                all_anomaly_results[model_name]["max"][index] = \
                    max(score, all_anomaly_results[model_name]["max"][index])
                index += 1


def compare_all_results():
    for model_name in all_anomaly_results:
        to_be_printed = " " * (25 - len(model_name)) + model_name
        dataset_names = ["Rademacher", "SVHN", "CIFAR-100", "Average"]
        for name in dataset_names:
            to_be_printed += " | " + " "*(6-math.ceil(len(name)/2)) + \
                                name + " "*(6-math.floor(len(name)/2))

        print(to_be_printed)
        print("=" * (25 + len(dataset_names) * 15))

        get_results_max(model_name = model_name)
        for key in all_anomaly_results[model_name].keys():
            if (key != "max"):
                to_be_printed = " "*(25-len(key)) + key
                index = 0
                for result in all_anomaly_results[model_name][key]:
                    if (all_anomaly_results[model_name]["max"][index] == result):
                        result = "*" + '{:.2f}'.format(round(result * 100, 2)) + "%"
                    else:
                        result = '{:.2f}'.format(round(result * 100, 2)) + "%"
                    to_be_printed += " | " + " "*(6-math.ceil(len(result)/2)) + \
                                        result + " "*(6-math.floor(len(result)/2))
                    index += 1
                print(to_be_printed)
        print()

    print("\n* highlights the maximum AUROC Score for an OOD Dataset")

compare_all_results()

            default_model |  Rademacher  |     SVHN     |  CIFAR-100   |   Average   
                Max Logit |    70.85%    |    90.40%    |    87.45%    |    82.90%   
              Max Softmax |    80.25%    |    92.50%    |   *89.25%    |    87.33%   
            Cross Entropy |    70.50%    |    90.55%    |    86.65%    |    82.57%   
                      ViM |   *95.95%    |   *93.25%    |    83.50%    |   *90.90%   


* highlights the maximum AUROC Score for an OOD Dataset


## Data Augmentation

Now we will see if models trained using data augmentation methods for robustness help in OOD detection. 

We will load a CIFAR-10 model that used PixMix data augmentation during training, and see how it fares compared to the default model that did not use data augmentation.

In [ ]:
!gdown 1mjIfbb3mfXXvAZ1sBnjotFr5yYFmLi68 # Downloading PixMix model
!gdown 1skZT6yplO-Sv4M8Ksgzx14cTY3H-HgOa # Downlaoding wideresnet class

Downloading...
From: https://drive.google.com/uc?id=1mjIfbb3mfXXvAZ1sBnjotFr5yYFmLi68
To: /content/checkpoint.pth.tar
100% 71.8M/71.8M [00:00<00:00, 99.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1skZT6yplO-Sv4M8Ksgzx14cTY3H-HgOa
To: /content/wideresnet_with_pen.py
100% 4.03k/4.03k [00:00<00:00, 7.61MB/s]


In [ ]:
from wideresnet_with_pen import WideResNet as WideResNet2  # Load pre-trained model

In [ ]:
# Loading the PixMix model

pixmix_net = WideResNet2(depth=40, num_classes=10, widen_factor=4, drop_rate=0.3)
pixmix_net = torch.nn.DataParallel(pixmix_net)

if torch.cuda.is_available():
    checkpoint = torch.load('checkpoint.pth.tar')
    pixmix_net.load_state_dict(checkpoint['state_dict'])
    net.eval()
    net.cuda()
else:
    checkpoint = torch.load('checkpoint.pth.tar', map_location=torch.device('cpu'))
    pixmix_net.load_state_dict(checkpoint['state_dict'])
    net.eval()

## PixMix Model Testing

Let us now test the PixMix model with the same anomaly score calculators we coded before.

In [ ]:
print("======= Max Logit AUROC Scores =======")
print_all_results(max_logit_anomaly_score, "Max Logit", pixmix_net, model_name = "pixmix_trained_model")

======= Max Logit AUROC Scores =======
Rademacher Noise Detection
AUROC: 			94.80%

SVHN Detection
AUROC: 			92.75%

CIFAR-100 Detection
AUROC: 			85.90%


In [ ]:
print("======= Max Softmax Probability AUROC Scores =======")
print_all_results(max_softmax_anomaly_score, "Max Softmax Probability", pixmix_net, model_name = "pixmix_trained_model")

======= Max Softmax Probability AUROC Scores =======
Rademacher Noise Detection
AUROC: 			93.00%

SVHN Detection
AUROC: 			91.80%

CIFAR-100 Detection
AUROC: 			88.70%


In [ ]:
print("======= Cross Entropy AUROC Scores =======")
print_all_results(cross_entropy_anomaly_score, "Cross Entropy", pixmix_net, model_name = "pixmix_trained_model")

======= Cross Entropy AUROC Scores =======
Rademacher Noise Detection
AUROC: 			94.25%

SVHN Detection
AUROC: 			92.40%

CIFAR-100 Detection
AUROC: 			88.15%


In [ ]:
print("======= ViM_anomaly_score_calculator =======")
w, b = pixmix_net.module.fc.weight.cpu().detach().numpy(), pixmix_net.module.fc.bias.cpu().detach().numpy()
u = -np.matmul(pinv(w), b)
principal_space, alpha = compute_ViM_principal_space_and_alpha(cifar10_loader, pixmix_net)
print_all_results(ViM_anomaly_score_calculator, "ViM", pixmix_net, model_name = "pixmix_trained_model", use_penultimate = True)

======= ViM_anomaly_score_calculator =======
(200, 10)
Rademacher Noise Detection
AUROC: 			88.70%

SVHN Detection
AUROC: 			90.05%

CIFAR-100 Detection
AUROC: 			90.15%


## Compare No-data-augmentation Model vs PixMix Model

Let us now compare how the default model compared to the PixMix model by running the following cell. 

As evident in the result, the PixMix model successfully helps us in OOD detection, and has a higher AUROC score. This shows the phenomenon that models that are more adversarially robust have better OOD detection capabilities.

In [ ]:
compare_all_results()

            default_model |  Rademacher  |     SVHN     |  CIFAR-100   |   Average   
                Max Logit |    70.85%    |    90.40%    |    87.45%    |    82.90%   
              Max Softmax |    80.25%    |    92.50%    |   *89.25%    |    87.33%   
            Cross Entropy |    70.50%    |    90.55%    |    86.65%    |    82.57%   
                      ViM |   *95.95%    |   *93.25%    |    83.50%    |   *90.90%   

     pixmix_trained_model |  Rademacher  |     SVHN     |  CIFAR-100   |   Average   
                Max Logit |   *94.80%    |   *92.75%    |    85.90%    |    91.15%   
  Max Softmax Probability |    93.00%    |    91.80%    |    88.70%    |    91.17%   
            Cross Entropy |    94.25%    |    92.40%    |    88.15%    |   *91.60%   
                      ViM |    88.70%    |    90.05%    |   *90.15%    |    89.63%   


* highlights the maximum AUROC Score for an OOD Dataset
